<a href="https://colab.research.google.com/github/warren00857020/DeepLearning/blob/main/DeepLearning1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
import os
import cv2
import random
import numpy as np
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
data_dir = 'flowers'
categories = ['daisy', 'dandelion', 'rose', 'sunflower', 'tulip']
label_dict = {categories[i]: i for i in range(len(categories))}
img_size = 128
data = []
target = []
#需要將圖像轉換成數字矩陣，然後對其進行歸一化處理
for category in categories:
folder_path = os.path.join(data_dir, category)
img_names = os.listdir(folder_path)
for img_name in img_names:
    img_path = os.path.join(folder_path, img_name)
    img = cv2.imread(img_path)
    try:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (img_size, img_size))
        data.append(resized)
        target.append(label_dict[category])

    except Exception as e:
        print("Exception:", e)
#接下來 將資料轉換成numpy數組 並進行歸一化處理(將像素值縮放到0到1之間)
data = np.array(data)/255.0
data = np.reshape(data, (data.shape[0], img_size, img_size, 1))
target = np.array(target)

new_target = np_utils.to_categorical(target)

data, new_target = shuffle(data, new_target, random_state=42)

#建立模型 使用卷積神經網路
model = Sequential()

#新增一個卷積層，使用32個filter，3x3的kernel size，'relu'作為activation function
model.add(Conv2D(32, (3, 3), input_shape=data.shape[1:], activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#新增一個卷積層，使用64個filter，3x3的kernel size，'relu'作為activation function
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#新增一個卷積層，使用128個filter，3x3的kernel size，'relu'作為activation function
model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#新增一個卷積層，使用256個filter，3x3的kernel size，'relu'作為activation function
model.add(Conv2D(256, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

#Flatten
model.add(Flatten())

#新增一個全連接層，使用512個神經元，'relu'作為activation function
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))

#新增一個全連接層，使用256個神經元，'relu'作為activation function
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))

#新增一個final layer，使用5個神經元(分別代表每個類別)，'softmax'作為activation function
model.add(Dense(5, activation='softmax'))

#使用'adam'optimizer、'categorical_crossentropy'loss function和'accuracy'度量標準來編譯模型。
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

#使用我們的資料集對模型進行訓練
history = model.fit(data, new_target, epochs=50, validation_split=0.1)

#使用測試集來衡量模型的性能
test_data = []
test_target = []

for category in categories:
folder_path = os.path.join(data_dir, category)
img_names = os.listdir(folder_path)
for img_name in img_names:
    img_path = os.path.join(folder_path, img_name)
    img = cv2.imread(img_path)

    try:
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        resized = cv2.resize(gray, (img_size, img_size))
        test_data.append(resized)
        test_target.append(label_dict[category])

    except Exception as e:
        print("Exception:", e)
test_data = np.array(test_data)/255.0
test_data = np.reshape(test_data, (test_data.shape[0], img_size, img_size, 1))
test_target = np.array(test_target)

new_test_target = np_utils.to_categorical(test_target)

test_loss, test_acc = model.evaluate(test_data, new_test_target)
print('Test accuracy:', test_acc)

# 取得測試集的predictions
predictions = model.predict(test_data)

# 取得測試集的actual_labels
actual_labels = [np.argmax(i) for i in new_test_target]

# 取得預測的類別標籤
predicted_labels = [np.argmax(i) for i in predictions]

#印出confusion matrix and classification report
print(confusion_matrix(actual_labels, predicted_labels))
print(classification_report(actual_labels, predicted_labels))

#印出actual and predicted class labels for 10 random test images
for i in range(10):
  index = random.randint(0, len(test_data) - 1)
  print("Actual: ", categories[actual_labels[index]])
  print("Predicted: ", categories[predicted_labels[index]])
  plt.imshow(test_data[index], cmap='gray')
  plt.show()

In [11]:
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
import os